### Import Libraries

In [1]:
import pandas as pd #Install xrld dependency    
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import datetime as dt

import plotly.express as px
from jupyter_dash import JupyterDash

from dash import html, dcc
from dash.dependencies import Input, Output

from pathlib import Path

import csv

### Loading the raw data

In [2]:
path_cred_deb_df = ("C:\\Users\\Alejandro\\Desktop\\Alpha Project\\Code\\Credit_debit_report\\cred_deb.xls")
sheet_name = ("cant_tarj_cred_deb")
bcra_features = ['Cantidad de tarjetas de crédito titulares', 'Cantidad de tarjetas de crédito adicionales', 'Cantidad de tarjetas de débito']

From this dataset you could see that a series of steps had to be made in order to clean and use the data.

In [19]:
class BcraData():

    def __init__(self, excel_path: str, excel_tab: str, dataframe_names: list) -> object:
        """Extract the data from a csv file
        
        ### Inputs:
            excel_path (str): The path of the selected data.
            excel_tab (str): The excel tab from which the data will be obtained.
            dataframe_names (list): A list with the names of the dataframes.       
        """

        # Initizialing the variables.    
        self.excel_path = excel_path
        self.excel_tab = excel_tab
        self.dataframe_names = dataframe_names        
        
        # A list for each transformational state.
        self.listed_df = [] # Cleaned data with accumulative data
        self.difference_df_list= [] # Cleaned data with differential data
        self.stacked_df = [] # Stacked dataframes for each transformation.


    def get_data(self) -> pd.DataFrame: # get_data_frame
        """gets the data in a pandas datafarme format normalized and cleaned"""
        
        self.__extraction_transform()
        
        self.__accumulative_data()
        
        self.__calculate_difference()
        
        self.__merge_dfs()

        return self.merged_df


    # Dataframe extraction and cleaning process.
    def __extraction_transform(self)-> list:
        """For each feature to obtain it creates a dataframe from the original excel file."""
        
        for self.dataframe in self.dataframe_names:
            self.__data_frame_extraction()  # Extraction of the DataFrame from the excel file.
            self.__data_frame_cleaning()    # Cleans the datafarme from the non useful rows and columns.
            self.__columns_clean_rename()   # Renames the default columns names into the real ones.    
            self.__set_date_time()          # Transform the datafarme into a time series.
        
            self.listed_df.append(self.extracted_df)   # aggregates every Dataframe generated into a list.

        return self.listed_df



    # Dataframe extraction
    def __data_frame_extraction(self) -> pd.DataFrame:  
        """Takes the dataset from the excel file and transform it into a pandas DataFrame
        # Inputs:
        * excel_path (str): The path of the selected data.
        * excel_tab (str): The excel tab from which the data will be obtained.
        """

        self.extracted_df = pd.read_excel(self.excel_path,
                                         sheet_name = self.excel_tab,
                                         skiprows = 17,
                                         engine = "xlrd",
                                         header = [0,2,3,4])

        self.extracted_df = self.extracted_df.dropna(axis=0)
        
        return self.extracted_df



    def __data_frame_cleaning(self) -> list:
        """ Cleans the dataframe and remove the non useful elements
        """
              
        self.extracted_df = self.extracted_df[["Año", "Mes", self.dataframe]]
        # Slicing the data and droping the non useful levels.
        self.extracted_df = self.extracted_df.iloc[1:]
        self.extracted_df = self.extracted_df.droplevel(axis=1, level=[0,1,3])
        self.extracted_df = self.extracted_df.drop(columns=self.extracted_df.iloc[:, 0].name)

        return self.extracted_df




    def __columns_clean_rename(self) -> list:
        """It's cleans and rename the columns"""
        self.extracted_df = self.extracted_df.rename(columns={self.extracted_df.iloc[:, 0].name:"Mes",
                                                              self.extracted_df.iloc[:, 2].name:"Total",
                                                              self.extracted_df.iloc[:, 9].name:"Compañías financieras",
                                                              self.extracted_df.iloc[:, 10].name:"Cajas de crédito",
                                                              self.extracted_df.iloc[:, 1].name:"Año mes"})
        
        return self.extracted_df



    def __set_date_time(self) -> list:
        """ Transform the extracted data into a Time Series."""
        
        # transform into string every element in the column and split the follwings elements.
        month_list = list(self.extracted_df["Año mes"].values)
        month_strings = [str(months) for months in month_list]
        month_strings = [re.split("\.", months) for months in month_strings]

        # Creates a list from the months and years.
        month = [months[1] for months in month_strings]
        year = [years[0] for years in month_strings]
    
        # Creates the following 
        self.extracted_df["Month"] = month
        self.extracted_df["Year"] = year
        self.extracted_df["date"] = self.extracted_df["Month"] + "/" + self.extracted_df["Year"]
        self.extracted_df["date"] = pd.to_datetime(self.extracted_df["date"], format = "%m/%Y") + pd.offsets.MonthEnd()

        # set index an drop the not useful columns
        self.extracted_df = self.extracted_df.set_index("date")
        self.extracted_df = self.extracted_df.drop(columns=["Mes", "Año mes", "Month", "Year"])        

        return self.extracted_df
    
    
    # Accumulative data
    def __accumulative_data(self):
        self.__stacking_dfs(self.listed_df)
        self.accumulative_df = self.stacked_joined_df
        
        return self.accumulative_df


    # Transform the accumulative data into differential period by period observational data.
    def __calculate_difference(self) -> list:
        """it's transform from the data provided which shows an accumulative observations for each date into the difference within each one"""  
        
        for dataframe in range(len(self.listed_df)):
            self.columns_names = list(self.listed_df[dataframe].columns)
            self.listed_df[dataframe] = self.listed_df[dataframe].replace(["…"], 0)
            self.listed_df[dataframe] = self.listed_df[dataframe].astype("int32")            
            self.difference_df = self.listed_df[dataframe].diff(periods=1)
            
            for column in self.columns_names:
                self.difference_df[column] = self.difference_df[column].fillna(self.listed_df[dataframe].loc[self.listed_df[dataframe][column].index[0], column])

            self.difference_df_list.append(self.difference_df)
        
        self.__stacking_dfs(self.difference_df_list)
        
        self.difference_df = self.stacked_joined_df
        self.difference_df = self.difference_df.rename(columns={"cantidad_titulares":"cantidad_titulares_var"})

        return self.difference_df


    def __merge_dfs(self):
        
        self.accumulative_df = self.accumulative_df.reset_index()
        self.difference_df = self.difference_df.reset_index()
        
        
        self.merged_df = pd.merge(left = self.accumulative_df,
                                  right = self.difference_df,
                                  how="left"
                                  )
        self.merged_df = self.merged_df.set_index("date")
        
        return self.merged_df
    

    # Stacks the features into one
    def __stacking_dfs(self, list_dataframe:list) -> pd.DataFrame.stack:
        
        self.stacked_dfs = []

        for dataframe in range(len(list_dataframe)):
            
            list_dataframe[dataframe] = list_dataframe[dataframe].replace(["…"], 0)
            list_dataframe[dataframe] = list_dataframe[dataframe].astype("int32")
            
            df = list_dataframe[dataframe].stack().to_frame()
            df = df.rename(columns={0: self.dataframe_names[dataframe]})

            self.stacked_dfs.append(df)
        self.__join_dfs()
        self.__stack_features()
        
        return self.stacked_joined_df


    def __join_dfs(self) -> pd.DataFrame.join:
        self.stacked_dfs = self.stacked_dfs[0].join(self.stacked_dfs[1:])
        self.stacked_dfs[self.stacked_dfs.columns] = self.stacked_dfs[self.stacked_dfs.columns].replace(["…"], 0)
        self.stacked_dfs = self.stacked_dfs[self.stacked_dfs.columns].astype(int)
        self.stacked_dfs = self.stacked_dfs.reset_index()
        self.stacked_dfs = self.stacked_dfs.set_index(["date","level_1"])

        return self.stacked_dfs

    def __stack_features(self) -> pd.DataFrame.stack:
        self.stacked_dfs = self.stacked_dfs.stack().reset_index()
        self.stacked_dfs = self.stacked_dfs.set_index("date")
        self.stacked_dfs = self.stacked_dfs.rename(columns={"level_1":"entidades_financieras",
                                                          "level_2":"instrumento",
                                                          0:"cantidad_titulares"})
                                
        self.stacked_dfs["instrumento"] = self.stacked_dfs["instrumento"].str.split("Cantidad de ", expand=True)[1]
        
        self.stacked_joined_df = self.stacked_dfs

        return self.stacked_joined_df

    
    # Load into an csv file.
    def generate_csv(self, file_name:str) -> csv:
        
        self.merged_df.to_csv(file_name+".csv", sep=";", encoding='utf-8-sig')
        

In [ ]:
bcra_data = BcraData(excel_path = path_cred_deb_df, excel_tab = sheet_name, dataframe_names = bcra_features)

tarj_df = bcra_data.get_data()
bcra_data.generate_csv("final_cards")


tarj_df